In [93]:
import torch
from models.bert_for_attr import BertCrfForAttr
from processors.utils_attr import CNerTokenizer, get_entities
from transformers import BertTokenizer
from models.transformers import WEIGHTS_NAME,BertConfig
from processors.attr_seq import convert_examples_to_features

In [89]:
label_list = ["X", "B-a","I-a",'S-a','O',"[CLS]", "[SEP]"]
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for i, label in enumerate(label_list)}

config = BertConfig.from_pretrained("/Users/leo/projects/marvik/meli/development/BERT-Attribute-Value-Extract/outputs/attr_output/bert/config.json",
                                          num_labels=len(["X", "B-a","I-a",'S-a','O',"[CLS]", "[SEP]"]),
                                      cache_dir=None)

# tokenizer = CNerTokenizer.from_pretrained("bert-base-multilingual-cased")
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')


model = BertCrfForAttr.from_pretrained("/Users/leo/projects/marvik/meli/development/BERT-Attribute-Value-Extract/outputs/attr_output/bert/pytorch_model.bin",from_tf=False,
                                        config=config,cache_dir=None,
                                        label2id=label2id,device="cpu")

In [97]:
encoded_title = tokenizer("Xiaomi Mi Band 5 Smartwatch Original Cardio Caja Sellada ***** JAZAK SHOP *****\n\nXiaomi Mi Band 5 versión global  \n\n\nLa heredera de la Xiaomi Mi Band 4 trae novedades como la carga magnética, control del ciclo menstrual, pantalla OLED 1.1 más grande,\nmonitorización del sueño entre otras prestaciones.\n\nSu diseño y construcción permiten que sea un dispositivo sumergible hasta 50 metros de profundidad, y otra mejora esperada que algo tiene que ver con el diseño (y sobre todo con el uso) es la carga magnética. Desde Jazak Shop te contamos que con la Mi Band 5, a diferencia de con las anteriores, los usuarios no han de extraer el dispositivo de la correa para cargarlo, ya que basta con conectar un pin de carga magnética en la parte posterior del dispositivo, junto al sensor de frecuencia cardíaca, para que comience el proceso de carga.\nUn diseño conocido con un 20% más de contenido, felices en Jazak Shop de poder ofrecertela.\n\nSe mantiene el diseño alargado del núcleo de la pulsera, con ese nuevo panel AMOLED de 1,1 pulgadas que permite ver un 20% más de contenido que el anterior.\n\n\nNuevas funciones: más actividades y control del ciclo menstrual\nEn cuanto al software, se confirman los nuevos 11 modos deportivos profesionales (máquina elíptica, yoga, máquina de remar, bicicleta estática, comba, cinta de correr, correr, bicicleta, natación, andar y entrenamiento libre) y también el nuevo modo de salud para las mujeres, orientado a ser útil para el control del ciclo menstrual. También dispone de la función de Inteligencia de Actividad Personal (PAI), una métrica que convierte los datos de frecuencia cardíaca en una puntuación única y personal para evaluar la actividad diaria.\nEn Jazak Shop estamos felicies de traerte la nueva Mi Band de Xiaomi\n\nXiaomi Mi Band 5\nLa Mi Band 5 de Xiaomi puede emparejarse vía bluetooth con móviles iOS y Android que han añadido en ambas versiones es la función de control remoto para hacer fotos, a modo de disparador a distancia.Jazak Shop te la trae.\n\nAutonomía: 14 días, dependiendo el uso.\n\nFicha técnica de la Xiaomi Mi Band 5\n\nPANTALLA\nOLED a color 1,1 pulgadas, 126 * 294 píxeles\n\nDIMENSIONES Y PESO\n47,2 x 18,5 x 12,4 mm / 11,9 gramos\n\nSENSORES\nFrecuencia cardíaca\n\nMEMORIA\n512 KB RAM\n16 MB\n\nRESISTENCIA AL AGUA\n50 m de profundidad\n\nAUTONOMÍA\nBatería 125 mAh\nHasta 14 días\n\nCONECTIVIDAD\nBluetooth 5.0\n\nCOMPATIBILIDAD\niOS 10.0\nAndroid 5.0\n\nEn la caja:\n\n- Mi Band 5\n- Cable de carga\n- Manual en español\n\n------------------------------------------------------------------\nSOMOS JAZAK SHOP\n\n6 MESES DE GARANTÍA\nTodos nuestros productos cuentan con 96 hs de prueba. En caso de falla del equipo el cambio es inmediato por otro nuevo.\n\nUBICACIÓN\nEstamos ubicados en el barrio de Chacarita, cerca del Parque Los Andes, de la Estación Dorrego del subte B y de la Estación Villa Crespo del tren San Martin.\n\nDIAS DE ATENCIÓN\nLunes a Viernes de 11 a 18Hs y los días Sabado de 10 a 14Hs\n\nMERCADOENVIOS\nLas entregas las puede realizar OCASA, Correo Argentino, OCA, Urbano, Andreani y otros también. Mercadoenvios utiliza diferentes transportes según la localidad que especifique el comprador. El correo que hará la entrega lo determina Mercadoenvios después de la compra.\nPodes ver la demora y si es gratis o tiene costo ingresando tu código postal donde estan las opciones de entrega al inicio de esta publicación.\nENVIO por MOTO en CABA: Podés recibir tu compra en el día (Elegir la opción “Retiro en el Domicilio del Vendedor”)\nCosto del envio en Capital Federal $180 / Días de lluvia 50% de recargo / *Consultá los valores de envío a GBA\n\n\nRespondemos las 24 hs, los 365 dias del año.\n\n\nJAZAK SHOP", return_tensors="pt")
encoded_attr = tokenizer("Marca", return_tensors="pt")

# encoded_title = tokenizer.tokenize("Celular Samsung S20 Plus con resoución 4k")
# encoded_attr = tokenizer.tokenize("Marca")

# torch.tensor([f.t_input_ids for f in features], dtype=torch.long)

# encoded_title = torch.Tensor([tokenizer.convert_tokens_to_ids(tokenizer.tokenize("Celular Samsung S20 Plus con resoución 4k"))])
# encoded_attr = torch.Tensor([tokenizer.convert_tokens_to_ids(tokenizer.tokenize("Marca"))])



inputs = {
    "t_input_ids": encoded_title["input_ids"],
    "a_input_ids": encoded_attr["input_ids"]
}

output = model(**inputs)
logits = output[0]
preds, _ = model.crf._obtain_labels(logits, id2label, [len(encoded_title["input_ids"]), len(encoded_attr["input_ids"])])
preds = preds[0][1:-1] # [CLS]XXXX[SEP]
label_entities = get_entities(preds, id2label, "bios")

Token indices sequence length is longer than the specified maximum sequence length for this model (1033 > 512). Running this sequence through the model will result in indexing errors


IndexError: index out of range in self

In [96]:
label_entities

[]

KeyError: 'Indexing with integers (to access backend Encoding for a given batch index) is not available when using Python based tokenizers'